In [15]:
import os
import pandas as pd
import shutil
import sys
import wget
import zipfile

In [4]:
os.environ['KAGGLE_USERNAME'] = "indenbomdmitry"
os.environ['KAGGLE_KEY'] = "d71056824e379aa1756815ce2a658476"
import kaggle

In [6]:
def clear_directory(path_to_dir: str):
    if os.path.exists(path_to_dir):
        shutil.rmtree(path_to_dir)
    os.makedirs(path_to_dir)

In [7]:
def download_dataset_from_kaggle(kaggle_dataset_path: str, out_directory: str):
    clear_directory(out_directory)
    kaggle.api.dataset_download_files(kaggle_dataset_path,
                                      path=out_directory,
                                      unzip=True,
                                      quiet=False)

In [16]:
def bar_progress(current, total, width=80):
    progress_message = "Downloading: %d%% [%d / %d] bytes" % (current / total * 100, current, total)
    # Don't use print() as it will print in new line every time.
    sys.stdout.write("\r" + progress_message)
    sys.stdout.flush()

def download_dataset_on_link(url, out_directory):
    clear_directory(out_directory)
    filename = wget.download(url,
                             out=out_directory,
                             bar=bar_progress)
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall(out_directory)
    os.remove(filename)

In [36]:
def add_sequence_to_lexicon(lexicon: list, text: str, max_length: int):
    seq = ""
    for word in text.split():
        if len(word) > max_length:
            continue
        if len(seq) + len(word) > max_length:
            lexicon.append(seq)
            seq = ""
        if seq == "":
            seq = word
        else:
            seq += ' ' + word

    lexicon.append(seq)

    return lexicon

In [33]:
def write_lexicon(lexicon: list, path_to_file):
    with open(path_to_file, 'w', encoding="utf-8") as the_file:
        the_file.write('\n'.join(lexicon))

#### Preparing lexicon of indian english names for handwritten names:

In [12]:
path_to_dir = "../data/lexicon-english-names"
download_dataset_from_kaggle("kanchitank/indian-names-male-and-female",
                             path_to_dir)

100%|██████████| 539k/539k [00:00<00:00, 10.1MB/s]

In [23]:
data = pd.read_csv(path_to_dir + "/Names_dataset.csv")
# Deleting empty rows
data = data.dropna()

In [37]:
texts = []
max_len = 40
for sequence in data["name"].to_list():
    add_sequence_to_lexicon(texts, str(sequence), max_len)

In [38]:
print("Number of texts is " + str(len(texts)))

Number of texts is 125200


In [39]:
write_lexicon(texts, path_to_dir + "/names_lexicon.txt")

#### Preparing lexicon of russian news for cyrillic texts: